In [11]:
# Import necessary libraries
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from category_encoders import OneHotEncoder
from sklearn.pipeline import Pipeline
import joblib

# Load the data
df = pd.read_csv('./data/raw_data.csv')

# Handle missing values
df.fillna({
    'Gender': df['Gender'].mode()[0],
    'Married': df['Married'].mode()[0],
    'Dependents': df['Dependents'].mode()[0],
    'Self_Employed': df['Self_Employed'].mode()[0],
    'ApplicantIncome': df['ApplicantIncome'].mean(),
    'CoapplicantIncome': df['CoapplicantIncome'].mean(),
    'LoanAmount': df['LoanAmount'].mean(),
    'Loan_Amount_Term': df['Loan_Amount_Term'].mode()[0],
    'Credit_History': df['Credit_History'].mode()[0]
}, inplace=True)

# Perform one-hot encoding
ohe = OneHotEncoder(
    use_cat_names=True, 
    cols=['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area', 'Loan_Status']
)
encoded_df = ohe.fit_transform(df)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define options for categorical variables
options_dependents = [{'label': str(i), 'value': i} for i in range(0, 4)]
options_education = [{'label': 'Graduate', 'value': 'Graduate'}, {'label': 'Not Graduate', 'value': 'Not Graduate'}]
options_self_employed = [{'label': 'Yes', 'value': 'Yes'}, {'label': 'No', 'value': 'No'}]
options_credit_history = [{'label': 'Yes', 'value': 1}, {'label': 'No', 'value': 0}]
options_property_area = [{'label': 'Urban', 'value': 'Urban'}, {'label': 'Semiurban', 'value': 'Semiurban'}, {'label': 'Rural', 'value': 'Rural'}]

# Define color scheme
colors = {
    'background': '#f9f9f9',  # Light gray background
    'text': '#333333',        # Dark gray text color
    'accent': '#4CAF50',      # Green accent color
    'button': '#FF5733',      # Orange button color
}

# Define the layout of the web application
app.layout = html.Div(style={'backgroundColor': colors['background'], 'padding': '20px'}, children=[
    html.H1("Loan Approval Prediction", style={'color': colors['text'], 'textAlign': 'center'}),
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Gender', style={'color': colors['text']}),
        dcc.Dropdown(
            id='gender-dropdown',
            options=[
                {'label': 'Male', 'value': 1},
                {'label': 'Female', 'value': 0}
            ],
            value=1
        ),
    ]),
    # Add dropdowns for other features...
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Dependents', style={'color': colors['text']}),
        dcc.Dropdown(
            id='dependents-dropdown',
            options=options_dependents,
            value=0
        ),
    ]),
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Education', style={'color': colors['text']}),
        dcc.Dropdown(
            id='education-dropdown',
            options=options_education,
            value=1
        ),
    ]),
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Self Employed', style={'color': colors['text']}),
        dcc.Dropdown(
            id='self-employed-dropdown',
            options=options_self_employed,
            value=0
        ),
    ]),
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Credit History', style={'color': colors['text']}),
        dcc.Dropdown(
            id='credit-history-dropdown',
            options=options_credit_history,
            value=1
        ),
    ]),
    html.Div(style={'marginBottom': '20px'}, children=[
        html.Label('Property Area', style={'color': colors['text']}),
        dcc.Dropdown(
            id='property-area-dropdown',
            options=options_property_area,
            value=1
        ),
    ]),
    
    html.Button('Predict', id='predict-button', n_clicks=0, style={'backgroundColor': colors['button'], 'color': '#FFFFFF', 'marginTop': '20px'}),
    
    html.Div(id='output-prediction', style={'marginTop': '20px', 'fontWeight': 'bold', 'fontSize': '18px', 'color': colors['accent'], 'textAlign': 'center'})
])

# Load Model
model = joblib.load('final_model.sav')

# Define callback function to handle user input and generate prediction
@app.callback(
    Output('output-prediction', 'children'),
    [Input('predict-button', 'n_clicks')],
    [dash.dependencies.State('gender-dropdown', 'value'),
     dash.dependencies.State('dependents-dropdown', 'value'),
     dash.dependencies.State('education-dropdown', 'value'),
     dash.dependencies.State('self-employed-dropdown', 'value'),
     dash.dependencies.State('credit-history-dropdown', 'value'),
     dash.dependencies.State('property-area-dropdown', 'value')]
)
def predict_loan_approval(n_clicks, gender, dependents, education, self_employed, credit_history, property_area):
    if n_clicks > 0:
        # Create a DataFrame based on the input values
        input_data = pd.DataFrame({
            'Gender': [gender],
            'Dependents': [dependents],
            'Education': [education],
            'Self_Employed': [self_employed],
            'Credit_History': [credit_history],
            'Property_Area': [property_area]
        })
        
        # Encode the input data using the same encoder used for training
        encoded_input = ohe.transform(input_data)
        
        # Make predictions
        prediction = model.predict(encoded_input)
        prediction_result = 'Approved' if prediction[0] == 1 else 'Rejected'
        return f"Prediction: {prediction_result}"

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[11], line 139, in predict_loan_approval(
    n_clicks=1,
    gender=0,
    dependents=1,
    education='Not Graduate',
    self_employed='No',
    credit_history=0,
    property_area='Urban'
)
    129 input_data = pd.DataFrame({
    130     'Gender': [gender],
    131     'Dependents': [dependents],
   (...)
    135     'Property_Area': [property_area]
    136 })
    138 # Encode the input data using the same encoder used for training
--> 139 encoded_input = ohe.transform(input_data)
        input_data =    Gender  Dependents     Education Self_Employed  Credit_History  \
0       0           1  Not Graduate            No               0   

  Property_Area  
0         Urban  
        ohe = OneHotEncoder(cols=['Gender', 'Married', 'Education', 'Self_Employed',
                    'Property_Area', 'Loan_Status'],
              use